In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

# Ecopetrol

In [6]:
indice_referencia = "ECOPETROL.CL"
df_indice = pd.read_csv(f"{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.0,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.006410,0.0,-0.006410,-0.006410
2,2024-01-03 00:00:00-05:00,0.027601,0.0,-0.027601,0.027601
3,2024-01-04 00:00:00-05:00,-0.006198,0.0,0.006198,0.000000
4,2024-01-05 00:00:00-05:00,0.016632,0.0,-0.016632,0.000000


## Metricas y ANOVA

In [7]:

# Supongamos que df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")


Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd -0.147437     0.185098 -1.066664
1     Buy_Mantein_Rnd -0.197879     0.278315 -0.890643
2     Media_Movil_Rnd  0.034492     0.278652 -0.055653
3  Revertion_Mean_Rnd -0.424042     0.138962 -3.411304

Resultados ANOVA:
ARR: F=1.131, p-value=0.335
Volatilidad: F=1.131, p-value=0.335
Sharpe: F=1.131, p-value=0.335


C:\Users\jumar\AppData\Local\Temp\ipykernel_22204\3783521943.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [8]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR_acum'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

# Graficar la evolución de las métricas acumulativas
fig = go.Figure()
for metric in ['ARR_acum', 'Volatilidad_acum', 'Sharpe_acum']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(x=df_temp['Date'], y=df_temp[metric], mode='lines', name=f"{strategy} - {metric}"))

fig.update_layout(
    title="Evolución Acumulativa de Métricas en el Tiempo",
    xaxis_title="Fecha",
    yaxis_title="Valor de la Métrica",
    legend_title="Estrategia y Métrica",
    template="plotly_dark"
)

fig.show()


C:\Users\jumar\AppData\Local\Temp\ipykernel_22204\1733184384.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# NUTRESA

In [14]:
indice_referencia = "NUTRESA.CL"
df_indice = pd.read_csv(f"{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.0,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.017333,0.0,-0.017333,0.000000
2,2024-01-03 00:00:00-05:00,0.015727,0.0,-0.015727,0.000000
3,2024-01-04 00:00:00-05:00,-0.012903,0.0,0.012903,-0.012903
4,2024-01-05 00:00:00-05:00,-0.020044,0.0,0.020044,-0.020044


## Metricas y ANOVA

In [15]:
# Supongamos que df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")

Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd  0.305993     0.294766  0.868461
1     Buy_Mantein_Rnd  1.150495     0.604075  1.821787
2     Media_Movil_Rnd -0.143060     0.605918 -0.318624
3  Revertion_Mean_Rnd -0.753035     0.433885 -1.850803

Resultados ANOVA:
ARR: F=3.365, p-value=0.018
Volatilidad: F=3.365, p-value=0.018
Sharpe: F=3.365, p-value=0.018


C:\Users\jumar\AppData\Local\Temp\ipykernel_27256\1903470467.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [16]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR_acum'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

# Graficar la evolución de las métricas acumulativas
fig = go.Figure()
for metric in ['ARR_acum', 'Volatilidad_acum', 'Sharpe_acum']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(x=df_temp['Date'], y=df_temp[metric], mode='lines', name=f"{strategy} - {metric}"))

fig.update_layout(
    title="Evolución Acumulativa de Métricas en el Tiempo",
    xaxis_title="Fecha",
    yaxis_title="Valor de la Métrica",
    legend_title="Estrategia y Métrica",
    template="plotly_dark"
)

fig.show()

C:\Users\jumar\AppData\Local\Temp\ipykernel_27256\3985800484.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# BANCOLOMBIA

In [3]:
indice_referencia = "PFBCOLOM.CL"
df_indice = pd.read_csv(f"{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.000000,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.019385,0.000000,0.019385,0.000000
2,2024-01-03 00:00:00-05:00,-0.006557,0.000000,-0.006557,0.000000
3,2024-01-04 00:00:00-05:00,0.064686,-0.064686,0.064686,0.064686
4,2024-01-05 00:00:00-05:00,0.029138,-0.029138,0.029138,0.029138


## Metricas y ANOVA

In [4]:
# Supongamos que df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")

Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd  0.085744     0.128263  0.278681
1     Buy_Mantein_Rnd  0.341164     0.232590  1.251831
2     Media_Movil_Rnd  0.075982     0.233277  0.111380
3  Revertion_Mean_Rnd -0.358852     0.119566 -3.419459

Resultados ANOVA:
ARR: F=2.819, p-value=0.038
Volatilidad: F=2.819, p-value=0.038
Sharpe: F=2.819, p-value=0.038


C:\Users\jumar\AppData\Local\Temp\ipykernel_22204\1903470467.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_indice['Date'] = pd.to_datetime(df_indice['Date'])


## Evolución Metricas

In [5]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR_acum'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

# Graficar la evolución de las métricas acumulativas
fig = go.Figure()
for metric in ['ARR_acum', 'Volatilidad_acum', 'Sharpe_acum']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(x=df_temp['Date'], y=df_temp[metric], mode='lines', name=f"{strategy} - {metric}"))

fig.update_layout(
    title="Evolución Acumulativa de Métricas en el Tiempo",
    xaxis_title="Fecha",
    yaxis_title="Valor de la Métrica",
    legend_title="Estrategia y Métrica",
    template="plotly_dark"
)

fig.show()

C:\Users\jumar\AppData\Local\Temp\ipykernel_22204\3985800484.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)


# ARGOS

In [5]:
indice_referencia = "CEMARGOS.CL"
df_indice = pd.read_csv(f"{indice_referencia}/stock_sugerencias.csv")
df_indice=df_indice.loc[: ,["Date","Buy_Mantein_Rnd","Revertion_Mean_Rnd","Media_Movil_Rnd","Agente_BVC_Rnd"]]
df_indice.head(5)

,Date,Buy_Mantein_Rnd,Revertion_Mean_Rnd,Media_Movil_Rnd,Agente_BVC_Rnd
0,2024-01-01 00:00:00-05:00,0.000000,0.000000,0.000000,0.000000
1,2024-01-02 00:00:00-05:00,0.008183,0.000000,0.008183,0.000000
2,2024-01-03 00:00:00-05:00,-0.001623,0.000000,-0.001623,0.000000
3,2024-01-04 00:00:00-05:00,0.014634,0.000000,0.014634,0.014634
4,2024-01-05 00:00:00-05:00,0.030449,-0.030449,0.030449,0.030449


## Metricas y ANOVA

In [6]:
# Supongamos que df_indice ya está cargado con las columnas mencionadas
df_indice['Date'] = pd.to_datetime(df_indice['Date'])

# Transformar el DataFrame a un formato largo para análisis
df_melted = df_indice.melt(id_vars=['Date'], 
                            var_name='Estrategia', 
                            value_name='Rendimiento')

# Calcular métricas por estrategia
df_metrica = df_melted.groupby('Estrategia').agg(
    ARR=('Rendimiento', lambda x: ((1+np.mean(x)) ** (253)) - 1),
    Volatilidad=('Rendimiento', lambda x: np.std(x) * np.sqrt(253)),
    Sharpe=('Rendimiento', lambda x: ((((1+np.mean(x)) ** (253)) - 1)-0.05) / (np.std(x) * np.sqrt(253)))
).reset_index()

print("Métricas calculadas por estrategia:")
print(df_metrica)

# ANOVA para cada métrica
print("\nResultados ANOVA:")
for metric in ['ARR', 'Volatilidad', 'Sharpe']:
    groups = [df_melted[df_melted['Estrategia'] == strat]['Rendimiento'] for strat in df_melted['Estrategia'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"{metric}: F={f_stat:.3f}, p-value={p_value:.3f}")

Métricas calculadas por estrategia:
           Estrategia       ARR  Volatilidad    Sharpe
0      Agente_BVC_Rnd  0.134101     0.175430  0.479401
1     Buy_Mantein_Rnd  0.838392     0.305176  2.583399
2     Media_Movil_Rnd  0.264525     0.307219  0.698280
3  Revertion_Mean_Rnd -0.571957     0.189732 -3.278084

Resultados ANOVA:
ARR: F=6.040, p-value=0.000
Volatilidad: F=6.040, p-value=0.000
Sharpe: F=6.040, p-value=0.000


C:\Users\jumar\AppData\Local\Temp\ipykernel_19396\1903470467.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



## Evolución Metricas

In [7]:
# Transformar el DataFrame a formato largo
df_melted = df_indice.melt(id_vars=['Date'], var_name='Estrategia', value_name='Rendimiento')

# Función para calcular métricas acumulativas
def calcular_metricas_acumulativas(df):
    df = df.sort_values('Date').copy()  # Asegurar orden cronológico
    df['ARR_acum'] = (1 + df['Rendimiento']).cumprod() ** (253 / df.index.to_series().diff().fillna(1).cumsum()) - 1
    df['Volatilidad_acum'] = df['Rendimiento'].expanding().std() * np.sqrt(252)
    df['Sharpe_acum'] = (df['Rendimiento'].expanding().mean() / df['Rendimiento'].expanding().std()) * np.sqrt(252)
    return df

# Aplicar función por estrategia
df_time_metrics = df_melted.groupby('Estrategia', group_keys=False).apply(calcular_metricas_acumulativas)

# Graficar la evolución de las métricas acumulativas
fig = go.Figure()
for metric in ['ARR_acum', 'Volatilidad_acum', 'Sharpe_acum']:
    for strategy in df_time_metrics['Estrategia'].unique():
        df_temp = df_time_metrics[df_time_metrics['Estrategia'] == strategy]
        fig.add_trace(go.Scatter(x=df_temp['Date'], y=df_temp[metric], mode='lines', name=f"{strategy} - {metric}"))

fig.update_layout(
    title="Evolución Acumulativa de Métricas en el Tiempo",
    xaxis_title="Fecha",
    yaxis_title="Valor de la Métrica",
    legend_title="Estrategia y Métrica",
    template="plotly_dark"
)

fig.show()

C:\Users\jumar\AppData\Local\Temp\ipykernel_19396\3985800484.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

